In [ ]:
%pip install pandas
%pip install numpy 

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [3]:
df_prime = pd.read_csv("C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados/data_prime_video.csv")

In [4]:
df_prime

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Judgment Night,movie,"Action, Crime, Drama",1993.0,tt0107286,6.6,19960.0,NaN
1,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2381047.0,NaN
2,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,481447.0,NaN
3,Dancer in the Dark,movie,"Crime, Drama, Musical",2000.0,tt0168629,7.9,120724.0,NaN
4,The Dark,movie,"Drama, Fantasy, Horror",2005.0,tt0411267,5.3,11685.0,NaN
...,...,...,...,...,...,...,...,...
73146,NaN,tv,NaN,2023.0,NaN,NaN,NaN,NaN
73147,Tranquilo Papá,tv,Comedy,2017.0,tt7122438,5.6,21.0,NaN
73148,NaN,tv,NaN,2021.0,NaN,NaN,NaN,NaN
73149,Murder Drones,tv,"Action, Adventure, Animation",2021.0,tt15599734,7.8,3994.0,NaN


In [5]:
df_prime.describe()

,releaseYear,imdbAverageRating,imdbNumVotes
count,72943.000000,63949.000000,6.394900e+04
mean,2007.721111,5.952558,1.292447e+04
std,18.989614,1.372682,7.423526e+04
min,1902.000000,1.000000,5.000000e+00
25%,2004.000000,5.100000,9.800000e+01
50%,2015.000000,6.100000,4.350000e+02
75%,2020.000000,7.000000,2.286000e+03
max,2027.000000,10.000000,3.046940e+06


In [7]:
df_prime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73151 entries, 0 to 73150
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               71372 non-null  object 
 1   type                73151 non-null  object 
 2   genres              70910 non-null  object 
 3   releaseYear         72943 non-null  float64
 4   imdbId              66262 non-null  object 
 5   imdbAverageRating   63949 non-null  float64
 6   imdbNumVotes        63949 non-null  float64
 7   availableCountries  282 non-null    object 
dtypes: float64(3), object(5)
memory usage: 4.5+ MB


In [8]:
df_prime.isnull().sum()

title                  1779
type                      0
genres                 2241
releaseYear             208
imdbId                 6889
imdbAverageRating      9202
imdbNumVotes           9202
availableCountries    72869
dtype: int64

In [10]:
df_prime.dropna(subset=['title'], inplace=True)

In [12]:
#preencher os NaN de genres com "Não informado"
df_prime['genres'] = df_prime['genres'].fillna('Não informado')

In [13]:
# os NaN em imdbId e imdbAverageRating podem ser um problema se for analisar a avaliação dos filmes, então preciso ver se o NaN em um coincide com NaN na outra

# Investigar a situação dos NaN
print("Registros sem imdbId:", df_prime['imdbId'].isna().sum())
print("Registros sem rating:", df_prime['imdbAverageRating'].isna().sum())

# Ver se NaN em imdbId implica em NaN em rating
mask_sem_id = df_prime['imdbId'].isna()
print("Dos que não têm ID, quantos também não têm rating:", 
      df_prime[mask_sem_id]['imdbAverageRating'].isna().sum())

Registros sem imdbId: 5110
Registros sem rating: 7423
Dos que não têm ID, quantos também não têm rating: 5110


In [14]:
# Investigar os casos com ID mas sem avaliação
tem_id_sem_rating = df_prime['imdbId'].notna() & df_prime['imdbAverageRating'].isna()
print("Registros com ID mas sem avaliação:", tem_id_sem_rating.sum())

# Ver alguns exemplos desses casos
print("\nExemplos de conteúdo com ID mas sem avaliação:")
exemplos = df_prime[tem_id_sem_rating][['title', 'imdbId', 'releaseYear']].head()
print(exemplos)

Registros com ID mas sem avaliação: 2313

Exemplos de conteúdo com ID mas sem avaliação:
             title      imdbId  releaseYear
1316        Rounds   tt8504330       2007.0
3278      Realtime   tt2333982       2007.0
9047   Becoming 13  tt10418724       2006.0
9067  Kokumon Gate  tt13174762       2019.0
9175      Goldfish  tt27432183       2023.0


In [15]:
#preencher os NaN de imdbid com "Não informado"
df_prime['imdbId'] = df_prime['imdbId'].fillna('Não informado')

In [16]:
def impute_ratings_by_genre_decade(df_prime):
    """
    Imputa valores NaN de imdbAverageRating usando mediana por gênero primário + década
    
    Args:
        df: DataFrame com colunas 'genres', 'releaseYear', 'imdbAverageRating'
    
    Returns:
        DataFrame com ratings imputados e colunas adicionais para rastreamento
    """
    # Extrair gênero primário (primeiro da lista)
    df_prime['primaryGenre'] = df_prime['genres'].fillna('Unknown').str.split(',').str[0].str.strip()

    # Extrair década
    df_prime['decade'] = (df_prime['releaseYear'] // 10) * 10

    # Flag para indicar se tinha rating original
    df_prime['hasOriginalRating'] = df_prime['imdbAverageRating'].notna()

    # Calcular medianas por gênero + década (apenas filmes COM rating)
    films_with_rating = df_prime[df_prime['hasOriginalRating']]
    
    medians_genre_decade = (films_with_rating.groupby(['primaryGenre', 'decade'])['imdbAverageRating'].agg(['median', 'count']).reset_index())

    # Calcular medianas só por gênero (fallback)
    medians_genre = (films_with_rating.groupby('primaryGenre')['imdbAverageRating'].median().reset_index())
    
    # Mediana global (último recurso)
    global_median = films_with_rating['imdbAverageRating'].median()

    # Função aninhada para imputar um filme específico
    def impute_single_film(row):
        if row['hasOriginalRating']:
            return pd.Series({
                'imdbAverageRating': row['imdbAverageRating'],
                'ratingImputed': False,
                'imputationSource': 'original'
            })
        
        # Tentar gênero + década primeiro
        genre_decade_match = medians_genre_decade[
            (medians_genre_decade['primaryGenre'] == row['primaryGenre']) & 
            (medians_genre_decade['decade'] == row['decade'])
        ]
        
        if not genre_decade_match.empty:
            median_val = genre_decade_match.iloc[0]['median']
            count = genre_decade_match.iloc[0]['count']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} {int(row['decade'])}s ({count} filmes)"
            })
        
        # Fallback: só gênero
        genre_match = medians_genre[medians_genre['primaryGenre'] == row['primaryGenre']]
        
        if not genre_match.empty:
            median_val = genre_match.iloc[0]['imdbAverageRating']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} geral"
            })
        
        # Último recurso: mediana global
        return pd.Series({
            'imdbAverageRating': global_median,
            'ratingImputed': True,
            'imputationSource': "mediana global"
        })

    # Aplicar imputação
    imputation_results = df_prime.apply(impute_single_film, axis=1)

    # Combinar resultados
    df_prime['imdbAverageRating'] = imputation_results['imdbAverageRating']
    df_prime['ratingImputed'] = imputation_results['ratingImputed']
    df_prime['imputationSource'] = imputation_results['imputationSource']

    return df_prime, medians_genre_decade, global_median

In [17]:
# APLICAR A IMPUTAÇÃO
df_imputed, medians_table, global_median = impute_ratings_by_genre_decade(df_prime)

In [18]:
imputed_count = df_imputed['ratingImputed'].sum()
print(f"Valores imputados: {imputed_count}")

Valores imputados: 7423


In [19]:
# Mostrar alguns exemplos organizados
for i, (_, filme) in enumerate(df_imputed.head(8).iterrows()):
    print(f"{i+1:2d}. \"{filme['title']}\" ({int(filme['releaseYear'])})")
    print(f"    Gênero: {filme['primaryGenre']}")
    print(f"    Avaliação imputada: {filme['imdbAverageRating']:.1f}")
    print(f"    Fonte: {filme['imputationSource']}")
    print()

 1. "Judgment Night" (1993)
    Gênero: Action
    Avaliação imputada: 6.6
    Fonte: original

 2. "Forrest Gump" (1994)
    Gênero: Drama
    Avaliação imputada: 8.8
    Fonte: original

 3. "Citizen Kane" (1941)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 4. "Dancer in the Dark" (2000)
    Gênero: Crime
    Avaliação imputada: 7.9
    Fonte: original

 5. "The Dark" (2005)
    Gênero: Drama
    Avaliação imputada: 5.3
    Fonte: original

 6. "Metropolis" (1927)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 7. "My Life Without Me" (2003)
    Gênero: Drama
    Avaliação imputada: 7.4
    Fonte: original

 8. "The Endless Summer" (1966)
    Gênero: Documentary
    Avaliação imputada: 7.6
    Fonte: original



In [21]:
# Axis 1 para colunas, axis 0 para linhas
df_prime = df_prime.drop(['availableCountries'], axis=1)

In [22]:
df_prime.isnull().sum()

title                   0
type                    0
genres                  0
releaseYear            92
imdbId                  0
imdbAverageRating       0
imdbNumVotes         7423
primaryGenre            0
decade                 92
hasOriginalRating       0
ratingImputed           0
imputationSource        0
dtype: int64

In [ ]:
# Ver quais são os filmes problemáticos
nulls_year = df_prime[df_prime['releaseYear'].isna()]
print(nulls_year[['title', 'type', 'genres', 'imdbId', 'imdbAverageRating']])

                                  title   type                     genres  \
28421                          Stranded  movie              Não informado   
35285  Paul Morrissey: A Real Humdinger  movie              Não informado   
35543      Moody McCarthy: The Good Jew  movie              Não informado   
35820  Keith Barany: Dude... Seriously?  movie              Não informado   
36107                 Scott Losse: Okay  movie              Não informado   
...                                 ...    ...                        ...   
63288                         The Reach  movie  Animation, Drama, Fantasy   
68984                       Charlemagne     tv                      Drama   
71668      3-nen Z-gumi Ginpachi Sensei     tv          Animation, Comedy   
72783     Untitled One Tree Hill Sequel     tv                      Drama   
72876                           Ferrari     tv                      Drama   

              imdbId  imdbAverageRating  
28421  Não informado             

In [29]:
df_prime = df_prime.dropna(subset=['releaseYear'])

In [30]:
df_prime['releaseYear'] = df_prime['releaseYear'].astype(int)

In [26]:
# ==========================================
# LIMPEZA FINAL - REMOVER BAGAGEM TÉCNICA
# ==========================================

# 1. TRATAR imdbNumVotes para filmes imputados
print("🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS")
print("=" * 50)

# Para filmes com rating imputado, vamos usar a mediana de votos do mesmo gênero/década
films_with_votes = df_prime[df_prime['imdbNumVotes'].notna()]

# Calcular mediana de votos por gênero primário
median_votes_by_genre = (films_with_votes
                        .groupby('primaryGenre')['imdbNumVotes']
                        .median()
                        .to_dict())

# Mediana global como fallback
global_median_votes = films_with_votes['imdbNumVotes'].median()

# Função para imputar votos
def impute_votes(row):
    if pd.notna(row['imdbNumVotes']):
        return row['imdbNumVotes']
    
    # Se tem rating imputado, usar mediana do gênero
    if row['ratingImputed']:
        genre_median = median_votes_by_genre.get(row['primaryGenre'])
        if genre_median:
            return genre_median
        else:
            return global_median_votes
    
    return row['imdbNumVotes']



🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS


In [27]:
# Aplicar imputação de votos
df_prime['imdbNumVotes'] = df_prime.apply(impute_votes, axis=1)


In [31]:
df_prime.isnull().sum()


title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
primaryGenre         0
decade               0
hasOriginalRating    0
ratingImputed        0
imputationSource     0
dtype: int64

In [32]:
df_prime['imdbNumVotes'] = df_prime['imdbNumVotes'].astype(int)
# Verificar se a conversão foi bem-sucedida
print(df_prime.dtypes)

title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
primaryGenre          object
decade               float64
hasOriginalRating       bool
ratingImputed           bool
imputationSource      object
dtype: object


In [33]:
# CRIAR DATASET LIMPO (apenas colunas essenciais) e removendo as colunas criadas para o tratamento
print(f"\n📂 CRIANDO DATASET LIMPO")
print("=" * 30)

colunas_essenciais = [
    'title', 
    'type', 
    'genres', 
    'releaseYear', 
    'imdbId', 
    'imdbAverageRating', 
    'imdbNumVotes'
]

df_prime_clean = df_prime[colunas_essenciais].copy()

print(f"Forma final do dataset: {df_prime_clean.shape}")
print(f"Colunas finais: {list(df_prime_clean.columns)}")




📂 CRIANDO DATASET LIMPO
Forma final do dataset: (71280, 7)
Colunas finais: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes']


In [34]:
# VERIFICAÇÃO FINAL
print(f"\n✅ VERIFICAÇÃO FINAL")
print("=" * 35)

print("Valores nulos por coluna:")
print(df_prime_clean.isnull().sum())

print(f"\nTipos de dados:")
print(df_prime_clean.dtypes)


✅ VERIFICAÇÃO FINAL
Valores nulos por coluna:
title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
dtype: int64

Tipos de dados:
title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
dtype: object


# ========Exploração Tipo e Gênero ===========

In [37]:
tipos_unicos = df_prime_clean['type'].unique()
print("Tipos únicos na coluna 'type':", tipos_unicos)

Tipos únicos na coluna 'type': ['movie' 'tv']


In [38]:
# Contando quantas produções existem de cada tipo
contagem_tipos = df_prime_clean['type'].value_counts()
print(f"\nDistribuição por tipo:")
for tipo, quantidade in contagem_tipos.items():
    percentual = (quantidade / len(df_prime_clean)) * 100
    print(f"{tipo}: {quantidade} produções ({percentual:.1f}%)")


Distribuição por tipo:
movie: 63416 produções (89.0%)
tv: 7864 produções (11.0%)


In [39]:
# EXPLORANDO OS PADRÕES DOS GENRES
print("\nPADRÕES E ESTRUTURA DOS GENRES")
# Amostras aleatórias para entender o formato
print("Exemplos de como os genres aparecem:")
sample_genres = df_prime_clean['genres'].dropna().sample(10).values
for i, genre in enumerate(sample_genres, 1):
    print(f"{i:2d}. {genre}")


PADRÕES E ESTRUTURA DOS GENRES
Exemplos de como os genres aparecem:
 1. Comedy
 2. Crime, Drama, Mystery
 3. Family, Fantasy, Musical
 4. Drama, Horror, Thriller
 5. Documentary
 6. Comedy, Music
 7. Action
 8. Comedy, Music
 9. Comedy
10. Documentary


In [42]:
# Função para extrair genres individuais
def extract_individual_genres(df_prime_clean):
    """Extrai todos os genres individuais de uma coluna que pode conter múltiplos genres"""
    all_genres = []
    
    for genres_str in df_prime_clean['genres'].dropna():
        # Remove espaços e divide por vírgulas (assumindo que é o separador mais comum)
        if pd.notna(genres_str):
            # Tratamento flexível para diferentes separadores
            genres_list = re.split(r'[,;|]', str(genres_str))
            # Limpa e adiciona à lista
            clean_genres = [genre.strip() for genre in genres_list if genre.strip()]
            all_genres.extend(clean_genres)
    
    return all_genres

In [43]:
# Extraindo todos os genres individuais
individual_genres = extract_individual_genres(df_prime_clean)

In [44]:
# Contando frequencias
genre_counts = Counter(individual_genres)
print(f"Total de genres únicos encontrados: {len(genre_counts)}")
print(f"Total de ocorrências de genres: {sum(genre_counts.values())}")

Total de genres únicos encontrados: 35
Total de ocorrências de genres: 140627


In [45]:
# TOP GENRES MAIS POPULARES
print("\n TOP 20 GENRES MAIS POPULARES")
top_genres = genre_counts.most_common(20)
for i, (genre, count) in enumerate(top_genres, 1):
    percentage = (count / sum(genre_counts.values())) * 100
    print(f"{i:2d}. {genre:<25} | {count:>6} ocorrências ({percentage:>5.2f}%)")



 TOP 20 GENRES MAIS POPULARES
 1. Drama                     |  30291 ocorrências (21.54%)
 2. Comedy                    |  17338 ocorrências (12.33%)
 3. Action                    |  10609 ocorrências ( 7.54%)
 4. Documentary               |   9367 ocorrências ( 6.66%)
 5. Thriller                  |   9183 ocorrências ( 6.53%)
 6. Horror                    |   8254 ocorrências ( 5.87%)
 7. Crime                     |   7993 ocorrências ( 5.68%)
 8. Romance                   |   7841 ocorrências ( 5.58%)
 9. Adventure                 |   6346 ocorrências ( 4.51%)
10. Mystery                   |   4563 ocorrências ( 3.24%)
11. Family                    |   4028 ocorrências ( 2.86%)
12. Animation                 |   3901 ocorrências ( 2.77%)
13. Fantasy                   |   3042 ocorrências ( 2.16%)
14. Sci-Fi                    |   2758 ocorrências ( 1.96%)
15. Biography                 |   2485 ocorrências ( 1.77%)
16. History                   |   2332 ocorrências ( 1.66%)
17. Music

In [46]:
df_prime_clean.to_csv('data_prime_video_clean', index=False)